In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('arxiv_papers.csv')

In [6]:
data

,title,abstract,authors,figures,pages,tables,pdf_link,primary_subject,secondary_subjects,submitted_journal,published_journal,keywords,date
0,Insights on Galaxy Evolution from Interpretabl...,Galaxy appearances reveal the physics of how t...,['John F. Wu'],4.0,10.0,2.0,arxiv.org/pdf/2501.00089,Astrophysics of Galaxies,['Machine Learning'],AAS Journals,NaN,"['Galaxies (573)', 'Astronomy image processing...",2025-01-03
1,Hunting for the candidates of Changing-Look Bl...,The changing-look blazars (CLBs) are the blaza...,"['Shi-Ju Kang', 'Shan-Shan Ren', 'Yong-Gang Zh...",7.0,13.0,3.0,arxiv.org/pdf/2501.00094,High Energy Astrophysical Phenomena,NaN,ApJ,NaN,['Active galactic nuclei (16) – Blazars (164) ...,2025-01-03
2,First unambiguous detection of ammonia in the ...,The newly accessible mid-infrared (MIR) window...,"['Mathilde Mâlin', 'Anthony Boccaletti', 'Clém...",6.0,18.0,4.0,arxiv.org/pdf/2501.00104,Earth and Planetary Astrophysics,NaN,A&A,NaN,"['Planetary systems', 'Stars: individual: GJ 5...",2025-01-03
3,Exchange of meteorites between the terrestrial...,The evolution of the orbits of bodies ejected ...,['S.I. Ipatov'],0.0,6.0,0.0,arxiv.org/pdf/2501.00134,Earth and Planetary Astrophysics,NaN,NaN,Modern astronomy: from the Early Universe to e...,['planets and satellites: terrestrial planets'],2025-01-03
4,Autoencoder Reconstruction of Cosmological Mic...,Enhanced modeling of microlensing variations i...,"['Somayeh Khakpash', 'Federica Bianco', 'Georg...",11.0,18.0,1.0,arxiv.org/pdf/2501.00153,Instrumentation and Methods for Astrophysics,NaN,The Astrophysical Journal,NaN,['- Interdisciplinary astronomy(804) 1'],2025-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17771,Reconstructing spatially-varying multiplicativ...,We present a quadratic estimator that detects ...,"['Konstantinos Tanidis', 'David Alonso', 'Lanc...",6.0,12.0,0.0,arxiv.org/pdf/2512.13679,Cosmology and Nongalactic Astrophysics,NaN,NaN,NaN,['Gravitational lensing: weak – cosmology: lar...,2025-12-16
17772,Bridging Simulations and EFT: A Hybrid Model o...,The Lyman-alpha (Lya) forest is a unique probe...,"['Roger de Belsunce', 'Boryana Hadzhiyska', 'M...",6.0,11.0,3.0,arxiv.org/pdf/2512.13681,Cosmology and Nongalactic Astrophysics,['Astrophysics of Galaxies'],Phys,NaN,[],2025-12-16
17773,"XID+PRIMA, II: Stepping Through Hyperspectral ...",The PRobe far-Infrared Mission for Astrophysic...,"['J. M. S. Donnellan', 'B. Pautasso', 'S. J. O...",16.0,20.0,4.0,arxiv.org/pdf/2512.13682,Instrumentation and Methods for Astrophysics,NaN,Open Journal of Astrophysics,NaN,[],2025-12-16
17774,Identification of periodicities with arbitrary...,Massive black hole binaries are expected to be...,"['Lorenzo Bertassi', 'Maria Charisi', 'Riccard...",14.0,13.0,5.0,arxiv.org/pdf/2512.13688,Astrophysics of Galaxies,NaN,NaN,NaN,['quasars: supermassive black holes – galaxies...,2025-12-16


In [15]:
import requests
import os

ADS_API_URL = "https://api.adsabs.harvard.edu/v1/search/query"

def get_citation_count(paper_title, max_results=5):
    """
    Query ADS for a paper by title and return citation count(s).
    """
    token = os.getenv("ADS_API_KEY")
    if token is None:
        raise RuntimeError("ADS_API_KEY environment variable not set")

    headers = {
        "Authorization": f"Bearer {token}"
    }

    params = {
        "q": f'title:"{paper_title}"',
        "fl": "title,author,year,citation_count,bibcode",
        "rows": max_results
    }

    response = requests.get(ADS_API_URL, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()
    papers = data["response"]["docs"]

    if not papers:
        print("No papers found.")
        return []

    results = []
    for p in papers:
        results.append({
            "title": p.get("title", [""])[0],
            "year": p.get("year"),
            "authors": ", ".join(p.get("author", [])[:3]) + (" et al." if len(p.get("author", [])) > 3 else ""),
            "citation_count": p.get("citation_count", 0),
            "bibcode": p.get("bibcode")
        })

    return results


In [16]:
paper_title = "Insights on Galaxy Evolution from Interpretable Sparse Feature Networks"

results = get_citation_count(paper_title)

for r in results:
    print(f"{r['title']} ({r['year']})")
    print(f"Authors: {r['authors']}")
    print(f"Citations: {r['citation_count']}")
    print(f"Bibcode: {r['bibcode']}")
    print("-" * 60)

Insights into Galaxy Evolution from Interpretable Sparse Feature Networks (2025)
Authors: Wu, John F.
Citations: 3
Bibcode: 2025ApJ...980..183W
------------------------------------------------------------


In [23]:
import requests
import os

ADS_API_URL = "https://api.adsabs.harvard.edu/v1/search/query"

def ads_query(q, fl, rows=2000):
    token = os.getenv("ADS_API_KEY")
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": q,
        "fl": fl,
        "rows": rows
    }
    r = requests.get(ADS_API_URL, headers=headers, params=params)
    r.raise_for_status()
    return r.json()["response"]["docs"]


def get_paper_info_by_title(title):
    results = ads_query(
        q=f'title:"{title}"',
        fl="bibcode,title,author"
    )
    if not results:
        raise ValueError("Paper not found")

    paper = results[0]
    return {
        "bibcode": paper["bibcode"],
        "authors": set(paper.get("author", []))
    }

def get_citing_papers(bibcode):
    return ads_query(
        q=f"citations({bibcode})",
        fl="bibcode,author"
    )

def count_self_citations(paper_title):
    target = get_paper_info_by_title(paper_title)
    target_authors = target["authors"]

    citing_papers = get_citing_papers(target["bibcode"])

    total_citations = len(citing_papers)
    self_citations = 0

    for paper in citing_papers:
        citing_authors = set(paper.get("author", []))
        if target_authors & citing_authors:
            self_citations += 1

    return {
        "total_citations": total_citations,
        "self_citations": self_citations,
        "non_self_citations": total_citations - self_citations
    }


In [24]:
title = "Insights on Galaxy Evolution from Interpretable Sparse Feature Networks"

stats = count_self_citations(title)

print(stats)


{'total_citations': 3, 'self_citations': 2, 'non_self_citations': 1}


In [25]:
def list_citing_papers(paper_title):
    # Get target paper info
    target = get_paper_info_by_title(paper_title)
    bibcode = target["bibcode"]

    # Get all citing papers
    citing_papers = get_citing_papers(bibcode)

    print(f"\nPapers citing:\n{paper_title}")
    print(f"Bibcode: {bibcode}")
    print(f"Total citations found: {len(citing_papers)}\n")

    for i, paper in enumerate(citing_papers, start=1):
        title = paper.get("title", [""])[0]
        authors = ", ".join(paper.get("author", [])[:3])
        year = paper.get("year", "—")
        bib = paper.get("bibcode", "")

        print(f"{i:4d}. {title} ({year})")
        print(f"     {authors}")
        print(f"     bibcode: {bib}\n")


In [26]:
list_citing_papers("Insights on Galaxy Evolution from Interpretable Sparse Feature Networks")


Papers citing:
Insights on Galaxy Evolution from Interpretable Sparse Feature Networks
Bibcode: 2025ApJ...980..183W
Total citations found: 3

   1.  (—)
     Euclid Collaboration, Walmsley, M., Huertas-Company, M.
     bibcode: 2025arXiv250315310E

   2.  (—)
     Ntampaka, Michelle, Ciprijanovic, A., Delgado, Ana Maria
     bibcode: 2025arXiv251009748N

   3.  (—)
     Wu, John F., Walmsley, Michael
     bibcode: 2025arXiv251023749W



In [27]:
def list_citing_papers_with_self_flag(paper_title):
    target = get_paper_info_by_title(paper_title)
    target_authors = target["authors"]
    bibcode = target["bibcode"]

    citing_papers = get_citing_papers(bibcode)

    print(f"\nCiting papers for:\n{paper_title}\n")

    for i, paper in enumerate(citing_papers, start=1):
        title = paper.get("title", [""])[0]
        year = paper.get("year", "—")
        authors = set(paper.get("author", []))
        bib = paper.get("bibcode", "")

        is_self = bool(target_authors & authors)
        tag = "[SELF]" if is_self else ""

        print(f"{i:4d}. {title} ({year}) {tag}")
        print(f"     bibcode: {bib}\n")


In [29]:
list_citing_papers_with_self_flag("Insights on Galaxy Evolution from Interpretable Sparse Feature Networks")


Citing papers for:
Insights on Galaxy Evolution from Interpretable Sparse Feature Networks

   1.  (—) 
     bibcode: 2025arXiv250315310E

   2.  (—) [SELF]
     bibcode: 2025arXiv251009748N

   3.  (—) [SELF]
     bibcode: 2025arXiv251023749W

